In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import cv2
from skimage.transform import resize
from skimage.io import imread
from sklearn.utils import shuffle
import random
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import warnings
warnings.filterwarnings("ignore", message="divide by zero encountered in double_scalars")


2023-03-16 22:02:11.565380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing Image Data 

In [94]:
Categories=['rock','paper','scissors']
train_dir = '/Users/hanyijia/Desktop/Midterm_Project/rps-split/train'
test_dir = '/Users/hanyijia/Desktop/Midterm_Project/rps-split/test'
val_dir = '/Users/hanyijia/Desktop/Midterm_Project/rps-split/val'

flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(train_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(32,32,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for train')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_train=df.iloc[:,:-1] #input data 
y_train=df.iloc[:,-1] #output data
x_train=x_train.sample(frac=1, random_state=42).reset_index(drop=True)
y_train=y_train.sample(frac=1, random_state=42).reset_index(drop=True)



flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(test_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(32,32,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for test')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_test=df.iloc[:,:-1] #input data 
y_test=df.iloc[:,-1] #output data

flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(val_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(32,32,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for val')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_val=df.iloc[:,:-1] #input data 
y_val=df.iloc[:,-1] #output data

loaded category:rock successfully for train
loaded category:paper successfully for train
loaded category:scissors successfully for train
loaded category:rock successfully for test
loaded category:paper successfully for test
loaded category:scissors successfully for test
loaded category:rock successfully for val
loaded category:paper successfully for val
loaded category:scissors successfully for val


In [8]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.284348,0.297354,0.313321,0.336375,0.348745,0.359453,0.364130,0.370838,0.378704,0.384568,...,0.362042,0.359737,0.354575,0.350273,0.344003,0.336455,0.329440,0.326312,0.321663,0.312047
1,0.304267,0.309151,0.315684,0.326809,0.334907,0.342741,0.345765,0.349956,0.353411,0.358536,...,0.367493,0.365742,0.360479,0.355597,0.352232,0.343676,0.338224,0.335972,0.331903,0.324436
2,0.317973,0.321166,0.327402,0.338706,0.344916,0.354000,0.358446,0.363010,0.367416,0.371537,...,0.365138,0.376308,0.378158,0.374654,0.371060,0.363793,0.357014,0.353654,0.348882,0.339495
3,0.434079,0.443543,0.454310,0.465256,0.473619,0.483946,0.487627,0.493068,0.495262,0.501782,...,0.494751,0.489472,0.481426,0.465708,0.433235,0.401956,0.390577,0.384184,0.377176,0.370118
4,0.319114,0.321382,0.328557,0.338685,0.343122,0.351620,0.354666,0.358548,0.362823,0.369092,...,0.376347,0.374983,0.370926,0.365676,0.360789,0.353402,0.346790,0.344862,0.340304,0.332514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,0.317458,0.324743,0.334259,0.344573,0.353778,0.363156,0.366968,0.372741,0.377429,0.384629,...,0.385077,0.382807,0.379498,0.377461,0.369716,0.360159,0.354411,0.346691,0.342160,0.338577
1527,0.317021,0.322916,0.331836,0.342479,0.349516,0.358961,0.364288,0.369521,0.375182,0.382394,...,0.369951,0.365180,0.355349,0.342983,0.332119,0.320491,0.315504,0.316368,0.315536,0.311280
1528,0.361844,0.364937,0.370878,0.381801,0.390075,0.397263,0.403214,0.409365,0.416429,0.422234,...,0.332812,0.334786,0.331869,0.342953,0.359435,0.361635,0.360172,0.362851,0.361314,0.353466
1529,0.421583,0.428312,0.434783,0.442705,0.450507,0.456454,0.457194,0.460205,0.463888,0.467979,...,0.416330,0.412189,0.407762,0.403442,0.396639,0.389534,0.387599,0.383056,0.377605,0.375555


In [9]:
y_train

0       1
1       0
2       0
3       2
4       1
       ..
1526    2
1527    2
1528    1
1529    2
1530    2
Name: Target, Length: 1531, dtype: int64

# Model Construction

## LDA
### Basic Metric

In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import roc_auc_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [52]:
# Basic Metric

model = LinearDiscriminantAnalysis()
X_lda = model.fit(x_train, y_train).transform(x_train)
y_pred= model.predict(x_train)
#model.decision_function(x_train)
confusion_matrix(y_train, y_pred)
print(classification_report(y_train, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       508
           1       0.99      0.99      0.99       498
           2       0.97      0.99      0.98       525

    accuracy                           0.99      1531
   macro avg       0.99      0.99      0.99      1531
weighted avg       0.99      0.99      0.99      1531



In [57]:
y_pred2 = model.predict(x_test)
#model.decision_function(x_test)
cm2 = confusion_matrix(y_test, y_pred2)

print(cm2)
print(classification_report(y_test, y_pred2))

[[79 17 13]
 [22 63 22]
 [33 18 62]]
              precision    recall  f1-score   support

           0       0.59      0.72      0.65       109
           1       0.64      0.59      0.61       107
           2       0.64      0.55      0.59       113

    accuracy                           0.62       329
   macro avg       0.62      0.62      0.62       329
weighted avg       0.62      0.62      0.62       329



### LDA  - lsqr & eigen

In [84]:
#Parameter tuning with GridSearchCV 


estimator_1 = LinearDiscriminantAnalysis(shrinkage='auto')
parameters_1 = {
    'solver': ('lsqr','eigen'),  #note svd does not run with shrinkage and models using it will be tuned separately
    'n_components': (1,2),
                   }
# with GridSearch
grid_search_lda_A = GridSearchCV(
    estimator=estimator_1,
    param_grid=parameters_1,
    scoring = 'accuracy',
    refit = True,
    n_jobs = -1,
    cv = 5
)

In [85]:
lda_A1=grid_search_lda_A.fit(x_train, y_train)
y_pred_1 =lda_A1.predict(x_train)
print(confusion_matrix(y_train, y_pred_1))
print(classification_report(y_train, y_pred_1))

[[452  18  38]
 [ 14 439  45]
 [ 22  14 489]]
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       508
           1       0.93      0.88      0.91       498
           2       0.85      0.93      0.89       525

    accuracy                           0.90      1531
   macro avg       0.90      0.90      0.90      1531
weighted avg       0.90      0.90      0.90      1531



In [86]:
print(grid_search_lda_A.best_params_)

{'n_components': 1, 'solver': 'lsqr'}


In [87]:

y_pred_2 =lda_A1.predict(x_test)
print(confusion_matrix(y_test, y_pred_2))
print(classification_report(y_test, y_pred_2))

[[85  7 17]
 [17 71 19]
 [22 16 75]]
              precision    recall  f1-score   support

           0       0.69      0.78      0.73       109
           1       0.76      0.66      0.71       107
           2       0.68      0.66      0.67       113

    accuracy                           0.70       329
   macro avg       0.71      0.70      0.70       329
weighted avg       0.70      0.70      0.70       329



## LDA ~ svd

In [90]:
estimator_2 = LinearDiscriminantAnalysis(solver='svd', )#note svd does not run with shrinkage and models using it will be tuned separately
parameters_2 = {
    'n_components': (1,2),
    'store_covariance' :(True, False),
                   }
# with GridSearch
grid_search_lda_B = GridSearchCV(
    estimator=estimator_2,
    param_grid=parameters_2,
    scoring = 'accuracy',
    refit = True,
    n_jobs = -1,
    cv = 5
)

In [91]:
lda_B1=grid_search_lda_B.fit(x_train, y_train)
y_pred_3 =lda_B1.predict(x_train)
print(confusion_matrix(y_train, y_pred_3))
print(classification_report(y_train, y_pred_3))

[[495   2  11]
 [  1 494   3]
 [  2   1 522]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       508
           1       0.99      0.99      0.99       498
           2       0.97      0.99      0.98       525

    accuracy                           0.99      1531
   macro avg       0.99      0.99      0.99      1531
weighted avg       0.99      0.99      0.99      1531



In [92]:
print(grid_search_lda_B.best_params_)

{'n_components': 1, 'store_covariance': True}


In [93]:
y_pred_4 =lda_A1.predict(x_test)
print(confusion_matrix(y_test, y_pred_4))
print(classification_report(y_test, y_pred_4))

[[85  7 17]
 [17 71 19]
 [22 16 75]]
              precision    recall  f1-score   support

           0       0.69      0.78      0.73       109
           1       0.76      0.66      0.71       107
           2       0.68      0.66      0.67       113

    accuracy                           0.70       329
   macro avg       0.71      0.70      0.70       329
weighted avg       0.70      0.70      0.70       329



# QDA

## Basic Metric


In [95]:

QDA_model_default = QuadraticDiscriminantAnalysis()
QDA_model_default.fit(x_train, y_train)


/usr/local/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [96]:

y_pred_QDA_default =QDA_model_default.predict(x_train)
print(confusion_matrix(y_train, y_pred_QDA_default))
print(classification_report(y_train, y_pred_QDA_default))


y_pred_QDA_default1 =QDA_model_default.predict(x_test)
print(confusion_matrix(y_test, y_pred_QDA_default1))
print(classification_report(y_test, y_pred_QDA_default1))

[[508   0   0]
 [  0 498   0]
 [  0   0 525]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       508
           1       1.00      1.00      1.00       498
           2       1.00      1.00      1.00       525

    accuracy                           1.00      1531
   macro avg       1.00      1.00      1.00      1531
weighted avg       1.00      1.00      1.00      1531

[[40 30 39]
 [30 43 34]
 [35 40 38]]
              precision    recall  f1-score   support

           0       0.38      0.37      0.37       109
           1       0.38      0.40      0.39       107
           2       0.34      0.34      0.34       113

    accuracy                           0.37       329
   macro avg       0.37      0.37      0.37       329
weighted avg       0.37      0.37      0.37       329

